In [2]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:

!pip -q install dash
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px

# Load the data using pandas
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv')

# Initialize the Dash app
app = dash.Dash(__name__)
app.config.suppress_callback_exceptions = True

# Set the title of the dashboard
#app.title = "Automobile Statistics Dashboard"
app.layout = html.Div(children=[html.H1("Automobile Statistics Dashboard",
                                style={'textAlign': 'center', 'color': '#503D36',
                                'font-size': 26}),
#outer division starts
     html.Div([
                   # First inner division for adding dropdown helper text for Selected Drive wheels
                    html.Div([
                            html.H2("Select Your Statistics", style={'margin-right': '2em'}),

                    #Radio items to select the region
                    #dcc.RadioItems(df["Region"].unique(), value ='NSW', id='region',inline=True)]),

                    dcc.RadioItems([{"label":"Recession Period Statistics","value": "recession"},
                                    {"label":"Yearly Statistics","value": "yearly"}], "yearly", id="stats", inline = True)]),
                    #               {"label":"Queensland","value": "QL"},
                    #               {"label":"South Australia","value": "SA"},
                    #               {"label":"Tasmania","value": "TA"},
                    #                {"label":"Victoria","value": "VI"},
                    #                {"label":"Western Australia","value": "WA"}],"NSW", id='region',inline=True)]),
                    #Dropdown to select year
                    html.Div([
                            html.H2('Year', style={'margin-right': '2em'}),
                        dcc.Dropdown(df.Year.unique(), value = df.Year.unique()[0], id = 'year')
                    ]),
#Second Inner division for adding 2 inner divisions for 2 output graphs
#TASK 2.3: Add two empty divisions for output inside the next inner division.
                    html.Div([
                    html.Div([ ], id='plot1', style={'width': '50%', 'display': 'inline-block'}),
                    html.Div([ ], id='plot2', style={'width': '50%', 'display': 'inline-block'}),
                    html.Div([ ], id='plot3', style={'width': '50%', 'display': 'inline-block'}),
                    html.Div([ ], id='plot4', style={'width': '50%', 'display': 'inline-block'})], style={'margin-right': '2em'}),
    ])
    #outer division ends
])

@app.callback([Output(component_id="plot1", component_property="children")],
              [Output(component_id="plot2", component_property="children")],
              [Output(component_id="plot3", component_property="children")],
              [Output(component_id="plot4", component_property="children")],
              [Input(component_id="stats", component_property="value"),
               Input(component_id="year", component_property="value")])

#TASK 2.5: Add the callback function.
#Place to define the callback function .
def statistic(input_stats, input_year):
  if input_stats == "yearly":
    # Yearly Automobile Sales Statistics
    # Yearly Average Automobile sales using line chart for the whole period.
    fig1 = px.line(x=df.groupby('Year').mean().index, y=df.groupby('Year').mean()["Automobile_Sales"], title="Yearly Average Automobile Sales")
    fig1.update_xaxes(title_text="Year") # Adding x-axis label
    fig1.update_yaxes(title_text="Average Sales") # Adding y-axis label

    # Total Monthly Automobile sales using line chart.
    fig2 = px.line(df[df['Year'] == input_year], x='Month', y='Automobile_Sales', title="Total Monthly Automobile Sales in {}".format(input_year))

    # Average Monthly Automobile sales of each vehicle type using bar chart.
    fig3 = px.bar(df[df['Year'] == input_year].groupby(['Vehicle_Type', 'Month']).mean().reset_index(), x="Month", y="Automobile_Sales", color="Vehicle_Type", title="Average Monthly Automobile Sales by Vehicle Type in {}".format(input_year))

    # Total Advertisement Expenditure for each vehicle using pie chart
    temp = df[df['Year'] == input_year].groupby(['Vehicle_Type']).sum()
    fig4 = px.pie(values=temp['Advertising_Expenditure'], names=temp.index, title="Total Advertisement Expenditure by Vehicle Type in {}".format(input_year))
  elif input_stats == "recession":
    # Average Automobile sales using line chart for the Recession Period using line chart.
    fig1 = px.line(df[df['Recession'] == 1].groupby('Year').mean(), x=df[df['Recession'] == 1].groupby('Year').mean().index, y='Automobile_Sales', title="Average Automobile Sales During Recession")
    fig1.update_xaxes(title_text="Year") # Adding x-axis label
    fig1.update_yaxes(title_text="Average Sales") # Adding y-axis label

    # Average number of vehicles sold by vehicle type using bar chart
    fig2 = px.bar(df[df['Recession'] == 1].groupby(["Vehicle_Type"]).mean(numeric_only=True).reset_index(), x='Vehicle_Type', y="Automobile_Sales", color='Vehicle_Type', title="Average Number of Vehicles Sold by Vehicle Type During Recession")

    # Total expenditure share by vehicle type during recession using pie chart
    temp = df[df['Recession'] == 1].groupby('Vehicle_Type').sum()
    fig3 = px.pie(values=temp['Advertising_Expenditure'], names=temp.index, title="Total Expenditure Share by Vehicle Type During Recession")

    # Effect of unemployment rate on vehicle type and sales using bar chart
    temp = df[(df['Recession'] == 1)].groupby(["unemployment_rate", "Vehicle_Type"]).mean().reset_index()
    fig4 = px.bar(temp, x="unemployment_rate", y="Automobile_Sales", color="Vehicle_Type", barmode="group", title="Effect of Unemployment Rate on Vehicle Type and Sales During Recession")

  return [dcc.Graph(figure=fig1), dcc.Graph(figure=fig2), dcc.Graph(figure=fig3), dcc.Graph(figure=fig4)]
if __name__ == '__main__':
    app.run(jupyter_mode='external')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 26.8 MB/s eta 0:00:00
Dash app running on:


<IPython.core.display.Javascript object>